In [42]:
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import spmatrix

In [43]:
adata_norm_exp = sc.read_h5ad("norm_exp.h5ad")
print(adata_norm_exp)

AnnData object with n_obs × n_vars = 5577480 × 300
    obs: 'gw', 'sample', 'region', 'H1_cluster', 'H2_cluster', 'H3_cluster', 'H1_annotation', 'H2_annotation', 'H3_annotation', 'area', 'layer'
    obsm: 'spatial'


/Users/cyluo/anaconda3/envs/xuyuEnv/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [111]:
area_of_interest = ['V1', 'V2', 'A-V1', 'A-V2', 'B-V1', 'B-V2']
adata_norm_exp_subset = adata_norm_exp[adata_norm_exp.obs['area'].isin(area_of_interest)].copy()
adata_V1V2 = adata_norm_exp_subset[adata_norm_exp_subset.obs['gw'].isin(['gw15'])].copy()
adata_V1V2 = adata_V1V2[adata_V1V2.obs['sample'].isin(['UMB1367'])].copy()
adata_V1V2 = adata_V1V2[adata_V1V2.obs['region'].isin(['O1'])].copy()
adata_V1V2 = adata_V1V2[adata_V1V2.obs['H1_annotation'].isin(['EN-Mig'])].copy()

def transform_value(value):
    if value == 'V1' or value == 'A-V1' or value == 'B-V1':
        return 'V1'
    elif value == 'V2' or value == 'A-V2' or value == 'B-V2':
        return 'V2'

# Create new column A based on column B
adata_V1V2.obs['area_new'] = adata_V1V2.obs['area'].apply(transform_value)

print(adata_V1V2.obs.head())

                       gw   sample region  H1_cluster  H2_cluster  H3_cluster  \
cell                                                                            
2277265800298100216  gw15  UMB1367     O1           3          16          83   
2277265800298100237  gw15  UMB1367     O1           3          16          82   
2277265800298100240  gw15  UMB1367     O1           3          16          83   
2277265800298100246  gw15  UMB1367     O1           3          16          83   
2277265800298100291  gw15  UMB1367     O1           3          16          80   

                    H1_annotation H2_annotation H3_annotation  area layer  \
cell                                                                        
2277265800298100216        EN-Mig         EN-L2       EN-L2-3  B-V2    l4   
2277265800298100237        EN-Mig         EN-L2       EN-IZ-3  B-V2    l4   
2277265800298100240        EN-Mig         EN-L2       EN-L2-3  B-V2    l4   
2277265800298100246        EN-Mig         EN-L2

In [112]:
areas_of_interest = ['V1', 'V2']
adata_V1 = adata_V1V2[adata_V1V2.obs['area_new'] == areas_of_interest[0]].copy()
adata_V2 = adata_V1V2[adata_V1V2.obs['area_new']== areas_of_interest[1]].copy()

In [113]:
print(adata_V1.X)

[[0.         0.         0.         ... 0.         0.         0.07575758]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.0862069 ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.02035623 0.03816794]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [114]:
dense_X = adata_V1.X
mean_expression = np.mean(dense_X, axis=0)

dense_X2 = adata_V2.X
mean_expression2 = np.mean(dense_X2, axis=0)

In [115]:
expression_threshold = 0  
expressed_genes_mask = dense_X > expression_threshold
num_cells_expressing = np.sum(expressed_genes_mask, axis=0)

expressed_genes_mask2 = dense_X2 > expression_threshold
num_cells_expressing2 = np.sum(expressed_genes_mask2, axis=0)

genes_expressed_per_cell = np.sum(expressed_genes_mask, axis=1)
total_expressed_cells = np.sum(genes_expressed_per_cell > 0)
percent_expressed = num_cells_expressing / total_expressed_cells * 100

genes_expressed_per_cell2 = np.sum(expressed_genes_mask2, axis=1)
total_expressed_cells2 = np.sum(genes_expressed_per_cell2 > 0)
percent_expressed2 = num_cells_expressing2 / total_expressed_cells2 * 100

In [116]:
gene_names = adata_V1.var_names
mean_expression_per_gene = pd.Series(mean_expression, index=gene_names)
percent_expressed_per_gene = pd.Series(percent_expressed, index=gene_names)
percent_expressed_per_gene = percent_expressed_per_gene.to_frame(name='expressed cells (%)')

gene_names2 = adata_V2.var_names
mean_expression_per_gene2 = pd.Series(mean_expression2, index=gene_names)
percent_expressed_per_gene2 = pd.Series(percent_expressed2, index=gene_names)
percent_expressed_per_gene2 = percent_expressed_per_gene2.to_frame(name='expressed cells (%)')

In [117]:
print(mean_expression_per_gene)
print(percent_expressed_per_gene)

HS3ST1      0.015913
CD9         0.000316
CBLN4       0.000536
CA12        0.002204
LGALS1      0.000389
              ...   
SORCS1      0.001617
ANGPT2      0.000358
PAX6        0.005303
POU3F4      0.002339
NIPBL-DT    0.013101
Length: 300, dtype: float32
          expressed cells (%)
HS3ST1              26.498035
CD9                  0.749018
CBLN4                0.773576
CA12                 3.929273
LGALS1               1.006876
...                       ...
SORCS1               3.511788
ANGPT2               1.092829
PAX6                11.272102
POU3F4               5.194008
NIPBL-DT            27.234774

[300 rows x 1 columns]


In [128]:
de_df = pd.read_csv('DEG_EN_Mig.csv')
# Sort by log fold change
sorted_de_df = de_df.sort_values(by=['log_fold_change', '-log10_pval'], ascending=[False, False])
# Select top 10/20 and bottom 10/20 genes
N = 20
top_genes = sorted_de_df.head(N)
bottom_genes = sorted_de_df.tail(N)
# Combine top and bottom genes
selected_genes = pd.concat([top_genes, bottom_genes])
print(selected_genes)

     Unnamed: 0       gene  log_fold_change          pvals      pvals_adj  \
3             3      OPCML         1.670981  4.309595e-150  1.292879e-148   
1             1     PRSS12         1.335705  9.504131e-271  9.504131e-269   
24           24      CPNE8         1.224906   5.370746e-13   2.335107e-12   
14           14       DIO2         1.197640   4.635129e-27   3.476347e-26   
6             6      FEZF2         1.078703   9.582220e-87   1.796666e-85   
64           64       LGR6         1.014866   1.006330e-03   2.141127e-03   
10           10  LINC01089         1.011572   1.207163e-40   1.248790e-39   
7             7      SMOC1         0.997914   2.149096e-85   3.792522e-84   
4             4      SCN3B         0.803540  9.556271e-141  2.606256e-139   
58           58       TPBG         0.744806   1.235640e-04   2.918834e-04   
20           20      TTYH2         0.736422   2.019620e-17   1.044631e-16   
15           15       SYBU         0.731538   1.312953e-22   8.951951e-22   

In [129]:
# Assuming mean_expression_per_gene is a pandas Series and selected_genes is a pandas DataFrame
selected_gene_names = selected_genes['gene']
filtered_mean_expression = mean_expression_per_gene[mean_expression_per_gene.index.isin(selected_gene_names)]
filtered_mean_expression2 = mean_expression_per_gene2[mean_expression_per_gene2.index.isin(selected_gene_names)]
filtered_percent_expressed_per_gene = percent_expressed_per_gene[percent_expressed_per_gene.index.isin(selected_gene_names)]
filtered_percent_expressed_per_gene2 = percent_expressed_per_gene2[percent_expressed_per_gene2.index.isin(selected_gene_names)]

In [130]:
df_mean_expression = filtered_mean_expression.to_frame(name='mean expression')
print(df_mean_expression.head())

df_mean_expression2 = filtered_mean_expression2.to_frame(name='mean expression')
print(df_mean_expression2.head())

       mean expression
TSHZ3         0.006921
NPY           0.000598
LGR6          0.000608
BEST3         0.000353
TTYH2         0.003199
       mean expression
TSHZ3         0.004184
NPY           0.001105
LGR6          0.000301
BEST3         0.000714
TTYH2         0.001922


In [131]:
len(df_mean_expression)

40

In [132]:
combined_df_V1 = df_mean_expression.join(filtered_percent_expressed_per_gene, how='inner')
combined_df_V2 = df_mean_expression2.join(filtered_percent_expressed_per_gene2, how='inner')
print(combined_df_V1)

           mean expression  expressed cells (%)
TSHZ3             0.006921            11.161591
NPY               0.000598             2.320727
LGR6              0.000608             0.834971
BEST3             0.000353             0.626228
TTYH2             0.003199             8.521611
SPOCK1            0.006885            10.694990
SPARCL1           0.000184             0.810413
FEZF2             0.017335            24.459725
SST               0.000463             1.706778
CUX1              0.009369            22.679273
PRSS12            0.020628            47.482809
SCN3B             0.036336            48.710707
INSM1             0.036947            64.390963
NEUROG1           0.000543             1.215619
PENK              0.001106             4.027505
CPNE8             0.001525             2.983792
PTK2B             0.003703             6.078094
FSTL5             0.001969             4.150295
SMOC1             0.008446            25.503438
PEX5L             0.003638             7

In [133]:
combined_df_V1['area'] = areas_of_interest[0]
combined_df_V2['area'] = areas_of_interest[1]

In [134]:
print(combined_df_V2.head)

<bound method NDFrame.head of            mean expression  expressed cells (%) area
TSHZ3             0.004184             6.673261   V2
NPY               0.001105             3.942488   V2
LGR6              0.000301             0.427456   V2
BEST3             0.000714             1.158724   V2
TTYH2             0.001922             5.140071   V2
SPOCK1            0.004296             6.468365   V2
SPARCL1           0.000452             1.692161   V2
FEZF2             0.008245            13.339457   V2
SST               0.000962             3.726993   V2
CUX1              0.017415            35.726145   V2
PRSS12            0.008224            23.216872   V2
SCN3B             0.020980            32.960045   V2
INSM1             0.022870            45.702476   V2
NEUROG1           0.001163             2.497615   V2
PENK              0.002773             8.824672   V2
CPNE8             0.000653             1.190518   V2
PTK2B             0.002281             3.539760   V2
FSTL5           

In [135]:
# Ensure the gene names are aligned and sorted
combined_df_V1 = combined_df_V1.sort_index()
combined_df_V2 = combined_df_V2.sort_index()

In [136]:
combined_df_V1 = combined_df_V1.loc[selected_genes['gene']]
combined_df_V2 = combined_df_V2.loc[selected_genes['gene']]
print(combined_df_V1)

           mean expression  expressed cells (%) area
OPCML             0.011867            23.624754   V1
PRSS12            0.020628            47.482809   V1
CPNE8             0.001525             2.983792   V1
DIO2              0.001949             6.802554   V1
FEZF2             0.017335            24.459725   V1
LGR6              0.000608             0.834971   V1
LINC01089         0.006929            12.929764   V1
SMOC1             0.008446            25.503438   V1
SCN3B             0.036336            48.710707   V1
TPBG              0.001183             1.890963   V1
TTYH2             0.003199             8.521611   V1
SYBU              0.004663            12.143910   V1
TSHZ3             0.006921            11.161591   V1
SLC35F2           0.001778             2.996071   V1
RORB-AS1          0.000599             0.699902   V1
INSM1             0.036947            64.390963   V1
PTK2B             0.003703             6.078094   V1
SPOCK1            0.006885            10.69499

In [137]:
combined_df_V1.to_csv("combined_df_V1.csv")
combined_df_V2.to_csv("combined_df_V2.csv")